In [543]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd

In [544]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,40 mins 29 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.10
H2O cluster version age:,3 months and 13 days !!!
H2O cluster name:,H2O_from_python_unknownUser_waku8u
H2O cluster total nodes:,1
H2O cluster free memory:,3.219 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [441]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv


In [586]:
import pandas as pd

train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
data_to_predict = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
submission = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [443]:
train.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by of...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfire...,1


In [444]:
data_to_predict.head(5)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the ..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [445]:
submission.head(5)

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [587]:
train = train[['text', 'target']]

train.columns = ['tweet', 'class']

In [588]:
data_to_predict = data_to_predict[['text']]

data_to_predict.columns = ['tweet']

In [448]:
import re

hashtags = re.compile(r"^#\S+|\s#\S+")
mentions = re.compile(r"^@\S+|\s@\S+")
urls1 = r"(?:\@|https?\://)\S+"
urls2 = r"(?:\@|http?\://)\S+"
urls_pat = r'|'.join((urls1, urls2))
urls = re.compile(urls_pat)

def process_text(text):
  text = hashtags.sub(' hashtag', text)
  text = mentions.sub(' entity', text)
  return text.strip().lower()
  
def match_expr(pattern, string):
  return not pattern.search(string) == None

def get_data_wo_urls(dataset):
    link_with_urls = dataset.tweet.apply(lambda x: match_expr(urls, x))
    return dataset[[not e for e in link_with_urls]]

In [589]:
train.tweet = train.tweet.apply(process_text)
train = get_data_wo_urls(train)
train.head(4)

,tweet,class
0,our deeds are the reason of this hashtag may allah forgive us all,1
1,forest fire near la ronge sask. canada,1
2,all residents asked to 'shelter in place' are being notified by of...,1
3,"13,000 people receive hashtag evacuation orders in california",1


In [450]:
train['tweet']

0           our deeds are the reason of this hashtag may allah forgive us all
1                                      forest fire near la ronge sask. canada
2       all residents asked to 'shelter in place' are being notified by of...
3               13,000 people receive hashtag evacuation orders in california
4       just got sent this photo from ruby hashtag as smoke from hashtag p...
                                        ...                                  
7594    a gas thing just exploded and i heard screams and now the whole st...
7602        a siren just went off and it wasn't the forney tornado warning ??
7605    on the flip side i'm at walmart and there is a bomb and everyone h...
7609    entity entity the out of control wild fires in california even in ...
7611    police investigating after an e-bike collided with a car in little...
Name: tweet, Length: 3607, dtype: object

In [590]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words = 'english')

X_train = train['tweet']
y_train = train['class']
X_train = tfidf_vectorizer.fit_transform(X_train)

X_test = data_to_predict['tweet']
X_test = tfidf_vectorizer.transform(X_test)

In [603]:
df1 = pd.DataFrame(X_train.toarray(), columns=tfidf_vectorizer.get_feature_names())
df1['clase']=list(y_train)

df2 = pd.DataFrame(X_test.toarray(), columns=tfidf_vectorizer.get_feature_names())

In [585]:
df1.shape

(3607, 8459)

In [605]:
from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(df1, test_size=0.2)

In [606]:
train = h2o.H2OFrame(X_train)
val = h2o.H2OFrame(X_val)
test = h2o.H2OFrame(df2)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [607]:
train.dim

[2885, 8460]

In [608]:
val.dim

[722, 8460]

In [609]:
test.dim

[3263, 8459]

In [611]:
train['clase']=train['clase'].asfactor()
val['clase']=val['clase'].asfactor()

In [458]:
x = train.columns
y = 'clase'
x.remove(y)

aml = H2OAutoML(max_runtime_secs = 600)
aml.train(x = x, y = y, training_frame = train, leaderboard_frame = val)

AutoML progress: |██████████████████████████████████████ (failed)


OSError: Job with key $03017f00000132d4ffffffff$_8c141f147938497d140cde791f14f9f failed with an exception: water.exceptions.H2OConcurrentModificationException: Rollups not possible, because Vec was deleted: $04ff06000000ffffffff$_9c48ac1070d0e07713c11dd2cbf144f3
stacktrace: 
water.exceptions.H2OConcurrentModificationException: Rollups not possible, because Vec was deleted: $04ff06000000ffffffff$_9c48ac1070d0e07713c11dd2cbf144f3
	at water.fvec.RollupStats.start(RollupStats.java:307)
	at water.fvec.Vec.startRollupStats(Vec.java:915)
	at water.fvec.Vec.startRollupStats(Vec.java:903)
	at water.fvec.Frame.bulkRollups(Frame.java:512)
	at water.fvec.Frame.means(Frame.java:531)
	at hex.Model.makeBigScoreTask(Model.java:1575)
	at hex.Model.predictScoreImpl(Model.java:1598)
	at hex.Model.score(Model.java:1454)
	at hex.Model.score(Model.java:1438)
	at hex.Model.score(Model.java:1394)
	at ai.h2o.automl.Leaderboard.addModels(Leaderboard.java:236)
	at ai.h2o.automl.Leaderboard.addModel(Leaderboard.j

In [ ]:
aml.leaderboard

In [ ]:
aml.leader

In [ ]:
preds = aml.leader.predict(test)

In [ ]:
preds = preds['predict']
preds

In [ ]:
preds_final = preds.as_data_frame()
preds_final

In [ ]:
submission['target'] = preds_final['predict']
submission

In [ ]:
submission.to_csv('submission.csv', index=False)